In [10]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.metrics import roc_auc_score
import os

In [11]:
models = ['lightGBM_3']
print models

['lightGBM_3']


In [12]:
df = pd.read_csv('../output/{0}/fold_1/oof_pred.csv'.format(models[0]))
target = df.is_attributed.values
n = len(df)
print n
print list(target).count(1)

15000000
37062


In [13]:
df = pd.read_csv('../output/{0}/fold_1/test_pred.csv'.format(models[0]))
ids = df.click_id.tolist()
n_test = len(df)
print n_test

18790469


##### Simple average

In [14]:
output_dir = '../simple_ave1'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for m in models:
    print "Checking the ave of " + m
    ans = np.zeros(n)
    testp = np.zeros(n_test)
    for i in range(1, 3):
        print i,
        df = pd.read_csv('../output/{0}/fold_{1}/oof_pred.csv'.format(m, i))
        tmp = np.array(df.pred.tolist())
        print roc_auc_score(target, tmp),
        ans += 1./2. * tmp
        df = pd.read_csv('../output/{0}/fold_{1}/test_pred.csv'.format(m, i))
        testp += 1./2. * np.array(df.pred.tolist())
    score = roc_auc_score(target, ans)
    print "\n SCORE=" + str(score)
    oof_df = pd.DataFrame({
        'is_attributed': list(target),
        'pred': list(ans)
    })
    oof_df = oof_df[['is_attributed', 'pred']]
    oof_df.to_csv(output_dir + '/' + m + '_oof_{SCORE}.csv'.format(SCORE=str(score)), index=False)
    test_df = pd.DataFrame({
        'click_id': ids,
        'is_attributed': testp
    })
    test_df['click_id'] = test_df['click_id'].astype(int)
    test_df = test_df[['click_id', 'is_attributed']]
    test_df.to_csv(output_dir + '/' + m + '_test.csv', index=False)

Checking the ave of lightGBM_3
1 0.977313749841 2 0.977280419459 
 SCORE=0.977644989916


### Exponential Average

In [15]:
delta = 1E-10
def sigmoid(x):
    return 1. / (np.exp(-x)+1.)
def upper(x):
    return min(x, 1.-delta)
def lower(x):
    return max(x, delta)
def rev(x):
    return -np.log(1./x - 1.)

In [16]:
output_dir = '../exponential_ave1'
print len(target)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for m in models:
    print "Checking the ave of " + m
    ans = np.zeros(n)
    testp = np.zeros(n_test)
    for i in range(1, 3):
        print i,
        df = pd.read_csv('../output/{0}/fold_{1}/oof_pred.csv'.format(m, i))
        df.pred = df.pred.apply(upper)
        df.pred = df.pred.apply(lower)
        df.pred = df.pred.apply(rev)
        tmp = np.array(df.pred.tolist())
        print roc_auc_score(target, sigmoid(tmp)),
        ans += 1./2. * tmp
        df = pd.read_csv('../output/{0}/fold_{1}/test_pred.csv'.format(m, i))
        df.pred = df.pred.apply(upper)
        df.pred = df.pred.apply(lower)
        df.pred = df.pred.apply(rev)
        testp += 1./2. * np.array(df.pred.tolist())
    ans = sigmoid(ans)
    testp = sigmoid(testp)
    score = roc_auc_score(target, ans)
    print "\n SCORE=" + str(score)
    oof_df = pd.DataFrame({
        'is_attributed': list(target),
        'pred': list(ans)
    })
    oof_df = oof_df[['is_attributed', 'pred']]
    oof_df.to_csv(output_dir + '/' + m + '_oof_{SCORE}.csv'.format(SCORE=str(score)), index=False)
    test_df = pd.DataFrame({
        'click_id': ids,
        'is_attributed': testp
    })
    test_df['click_id'] = test_df['click_id'].astype(int)
    test_df = test_df[['click_id', 'is_attributed']]
    test_df.to_csv(output_dir + '/' + m + '_test.csv', index=False)

15000000
Checking the ave of lightGBM_3
1 0.977313749841 2 0.977280419459 
 SCORE=0.977633547864


##### Grid Search

In [17]:
def gridSearch(vecs, reso=0.02):
    assert len(vecs) == 9
    coeff = np.zeros(9)
    coeff[0] = 1.
    score = roc_auc_score(target, np.dot(coeff, vecs))
    for c1 in np.arange(0, 1., reso):
        print c1,
        for c2 in np.arange(0, 1.-c1, reso):
            for c3 in np.arange(0, 1.-c1-c2, reso):
                for c4 in np.arange(0, 1.-c1-c2-c3, reso):
                    for c5 in np.arange(0, 1.-c1-c2-c3-c4, reso):
                        for c6 in np.arange(0, 1.-c1-c2-c3-c4-c5, reso):
                            for c7 in np.arange(0, 1.-c1-c2-c3-c4-c5-c6, reso):
                                for c8 in np.arange(0, 1.-c1-c2-c3-c4-c5-c6-c7, reso):
                                    c9 = 1.-c1-c2-c3-c4-c5-c6-c7-c8
                                    tmp_coeff = np.array([c1, c2, c3, c4, c5, c6, c7, c8, c9])
                                    tmp_score = roc_auc_score(target, np.dot(tmp_coeff, vecs))
                                    if tmp_score > score:
                                        coeff = tmp_coeff
                                        score = tmp_score
    return coeff, score